# Lab 5: 2x3 Slider Puzzle Solver

## Introduction
This notebook focuses on solving a 2x3 Slider Puzzle. The problem is modeled as a state-space search problem where each move of a tile results in a new state. The puzzle is to be solved by a non-autonomous utility-based agent, which seeks the least-cost path from an initial to a goal configuration.

You will see *uninformed search* algorithms like `breadth_first_search`, `depth_first_search` and `uniform_cost_search`, as well as *informed search* algorithms like `greedy_search` and `astar_search` in action. 


---

### Essential Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import queue
from copy import deepcopy
sns.set()

### Problem Representation
Each state of the 2x3 puzzle is represented as a 2D NumPy array.
The blank tile is represented by 0. The objective is to rearrange the tiles so that they appear in ascending order from 1 to 5, with the 0 tile in the bottom-right corner.

### Helper Functions

**print_state(state, moved_tile=None)**

- Purpose: Prints visual representation of 2x3 puzzle grid. 

**get_neighbor(state)**

- Purpose: Generates all valid next states of the puzzle by sliding a tile into the empty space (0).
How it works:
    - Locates the blank (0) tile.
    - Checks if it can move up, down, left, or right.
    - Returns a list of tuples:(action, resulting_state_after_move)

**move_tile(state, direction)**
- Purpose: Returns a new state by sliding the blank tile in a specified direction.

**state_to_tuple(state)**
- Purpose: Converts the 2D NumPy array state into a hashable tuple, so it can be:
    - Stored in dictionaries (came_from, cost_so_far) since NumPy arrays can't be used as dictionary keys due to being mutable/unhashable
    - Used in sets and priority queues

##### **Summary of Movement Directions**

| Direction | Description                            | Visual Example (relative to blank) |
|-----------|----------------------------------------|-------------------------------------|
| `up`      | Moves the blank space **upward**       | blank swaps with tile above      |
| `down`    | Moves the blank space **downward**     | blank swaps with tile below      |
| `left`    | Moves the blank space **leftward**     | blank swaps with tile to the left|
| `right`   | Moves the blank space **rightward**    | blank swaps with tile to the right|


In [ ]:
# Function to visualise state (2D np array)
def print_state(state, title="", moved_tile=None):
    if title:
        print(f"\n{title}")
    
    for i in range(2):
        row = []
        for j in range(3):
            val = state[i, j]
            row.append(" " if val == 0 else str(val))
        print(", ".join(row))

# Function to convert state (2D NP array) to tuple 
def state_to_tuple(state):
    return tuple(state.flatten())

# Function to get all valid next states  
def get_neighbors(state):
    neighbors = []
    x, y = np.argwhere(state == 0)[0]
    
    moves = {
        "up":    (x - 1, y),
        "down":  (x + 1, y),
        "left":  (x, y - 1),
        "right": (x, y + 1)
    }
    
    for action, (nx, ny) in moves.items():
        if 0 <= nx < 2 and 0 <= ny < 3:
            new_state = state.copy()
            new_state[x, y], new_state[nx, ny] = new_state[nx, ny], new_state[x, y]
            neighbors.append((action, new_state))
    
    return neighbors

### Defining Initial and Goal States 

In [25]:
INITIAL_STATE = np.array([
    [5, 4, 2], 
    [1, 3, 0]
])

GOAL_STATE = np.array([
    [1, 2, 3],
    [4, 5, 0]
])

print_state(INITIAL_STATE, "Initial State: ")
print_state(GOAL_STATE, "Goal State: ")


Initial State: 
5, 4, 2
1, 3,  

Goal State: 
1, 2, 3
4, 5,  


## Uninformed Search Algorithms

### Breadth First Search

**Uninformed Search** : Focusses on the graph structure and not the *cost from start* or *distance from goal*.

In [53]:
from collections import deque
# State inputs are 2D NumPy arrays 

# --- BFS Implementation --- #
def slidingPuzzleBFS_np(initial_state, goal_state):
    """
    Solve a 6-tile sliding puzzle (2x3 board) using Breadth-First Search (BFS).
    States are represented as NumPy arrays.
    """
    # Convert initial and goal states to tuples
    start = state_to_tuple(initial_state)
    target = state_to_tuple(goal_state)
    
    # If initial state is goal, return 0 moves
    if start == target:
        print("BFS: Already at goal!")
        return 0
    
    # Initialise things 
    visited = set([start])              # Set to track visited states
    q = deque([(initial_state, 0, [])]) # Queue for BFS (stores state, number of moves, path)
    states_explored = 0                 # counter for visited states

    # Perform BFS
    while q:
        state, moves, path = q.popleft()
        states_explored += 1

        # If the current state is the goal, return the number of moves and the solution path
        if state_to_tuple(state) == target:
            print(f"BFS solution found: {moves} moves")
            print(f"BFS total states explored: {states_explored}")
            
            # Print the solution path 
            print_state(initial_state, title="Step 0: Initial State")
            for step, (action, s) in enumerate(path):
                print_state(s, title=f"Step {step + 1}: Move empty space {action}")
            return moves
        
        # Explore all neighbors
        for action, neighbor_state in get_neighbors(state):
            tup = state_to_tuple(neighbor_state)
            if tup not in visited:
                visited.add(tup)
                q.append((neighbor_state, moves + 1, path + [(action, neighbor_state)]))
    
    # If no solution found, return -1
    print(f"BFS: Unsolvable. Total states explored: {states_explored}")
    return -1

# --- BFS Testing --- #

print("\n===== SLIDING PUZZLE BFS TEST =====")
bfs_moves = slidingPuzzleBFS_np(INITIAL_STATE, GOAL_STATE)


===== SLIDING PUZZLE BFS TEST =====
BFS solution found: 8 moves
BFS total states explored: 53

Step 0: Initial State
5, 4, 2
1, 3,  

Step 1: Move empty space left
5, 4, 2
1,  , 3

Step 2: Move empty space up
5,  , 2
1, 4, 3

Step 3: Move empty space left
 , 5, 2
1, 4, 3

Step 4: Move empty space down
1, 5, 2
 , 4, 3

Step 5: Move empty space right
1, 5, 2
4,  , 3

Step 6: Move empty space up
1,  , 2
4, 5, 3

Step 7: Move empty space right
1, 2,  
4, 5, 3

Step 8: Move empty space down
1, 2, 3
4, 5,  


### Depth First Search

**Uninformed Search** : Focusses on the graph structure and not the *cost from start* or *distance from goal*.

In [55]:
# --- DFS Implementation --- #

def slidingPuzzleDFS_np(initial_state, goal_state):

    # Convert initial and goal states to tuples 
    start = state_to_tuple(initial_state)
    target = state_to_tuple(goal_state)
    
    # If initial state = goal, return 0 moves
    if start == target:
        print("DFS: Already at goal!")
        return 0
    
    # Initialise things  to track visited states (using tuple for hashability)
    visited = set([start])          # Set to track visited states
    stack = [(initial_state, 0)]    # Stack for DFS (stores state and number of moves)
    states_explored = 0             # Counter for visited states

    # Perform DFS
    while stack:
        state, moves = stack.pop()
        states_explored += 1
        
        # If the current state is the goal, return the number of moves and the solution path
        if state_to_tuple(state) == target:
            print(f"DFS solution found: {moves}")
            print(f"DFS total states explored: {states_explored}")    
            print_state(initial_state, title="Step 0: Initial State")
            print_state(state, title=f"Step {moves}: Final State (Goal Reached)")
            return moves
        
        # Explore all neighbors
        for action, neighbor_state in get_neighbors(state):
            tup = state_to_tuple(neighbor_state)
            if tup not in visited:
                visited.add(tup)
                stack.append((neighbor_state, moves + 1))
    
    # If no solution is found, return -1
    print(f"DFS: Unsolvable. Total states explored: {states_explored}")
    return -1

# --- Run the Test --- #
print("\n===== SLIDING PUZZLE DFS TEST =====")
dfs_moves = slidingPuzzleDFS_np(INITIAL_STATE, GOAL_STATE)


===== SLIDING PUZZLE DFS TEST =====
DFS solution found: 138
DFS total states explored: 182

Step 0: Initial State
5, 4, 2
1, 3,  

Step 138: Final State (Goal Reached)
1, 2, 3
4, 5,  


### Uniform Cost Search

**Uninformed Search** : UCS considers the path cost (cost from start) when selecting the next state. It uses a priority queue and selects the state in the frontier with the lowest path cost 

In [ ]:
import heapq
import numpy as np

# --- UCS Implementation --- #

def uniform_cost_search(initial_state, goal_state): 

    # Convert the initial and goal states to tuples 
    start = state_to_tuple(initial_state)
    target = state_to_tuple(goal_state)
    
    # If the initial state is already the goal, return 0 moves
    if start == target:
        print("UCS: Already at goal!")
        return 0
    
    # Initialise things 
    visited = set([start])
    frontier = []  # Priority queue for UCS (cost, state, path)
    heapq.heappush(frontier, (0, start, []))  # (cost, state, path)
    states_explored = 0
    
    # Perform UCS 
    while frontier:
        cost, state_tup, path = heapq.heappop(frontier)  # Pop least cost state
        state = np.array(state_tup).reshape(2, 3)
        states_explored += 1
        
        # If the current state is the goal, print the solution path
        if state_tup == target:
            print(f"UCS solution found: {cost} moves")
            print(f"UCS total states explored: {states_explored}")
            
            # Print the solution path with instructions
            print_state(initial_state, title="Step 0: Initial State")
            for step, (action, s) in enumerate(path):
                print_state(s, title=f"Step {step + 1}: Move empty space {action}")
            
            return cost
        
        # Explore all neighbors
        for action, neighbor_state in get_neighbors(state):
            tup = state_to_tuple(neighbor_state)
            
            if tup not in visited:
                visited.add(tup)
                heapq.heappush(frontier, (cost + 1, tup, path + [(action, neighbor_state)]))  # Add neighbor to frontier
    
    # If no solution is found, return -1
    print("UCS: Unsolvable puzzle.")
    print(f"UCS: Total states explored: {states_explored}")
    return -1, []

# --- Run the Test --- #
print("\n===== SLIDING PUZZLE UCS TEST =====")
ucs_moves = uniform_cost_search(INITIAL_STATE, GOAL_STATE)



===== SLIDING PUZZLE UCS TEST =====
UCS solution found: 8 moves
UCS total states explored: 49

Step 0: Initial State
5, 4, 2
1, 3,  

Step 1: Move empty space left
5, 4, 2
1,  , 3

Step 2: Move empty space up
5,  , 2
1, 4, 3

Step 3: Move empty space left
 , 5, 2
1, 4, 3

Step 4: Move empty space down
1, 5, 2
 , 4, 3

Step 5: Move empty space right
1, 5, 2
4,  , 3

Step 6: Move empty space up
1,  , 2
4, 5, 3

Step 7: Move empty space right
1, 2,  
4, 5, 3

Step 8: Move empty space down
1, 2, 3
4, 5,  


## Informed Search Algorithms

### Heuristic Function with Manhatten Distance - For A*Star

In informed search algorithms, a heuristic function estimates the cost from the current state to the goal. One common and effective heuristic for grid-based problems is the Manhattan Distance. The Manhattan Distance is the total number of horizontal and vertical moves required to reach from one point to another in a grid.

For two points **A(x₁, y₁) and B(x₂, y₂) on a 2D grid**, the Manhattan Distance is calculated as:

**ManhattanDistance(A, B) = |x1-x2| + |y1-y2|**

#### In the context of a 2x3 sliding tile puzzle:

- Each tile has a goal position (where it should end up).
- The Manhattan Distance is calculated for each tile between its current position and its goal position.
- The heuristic value h(n) is the sum of all individual tile distances (excluding the blank tile).

This provides a good estimate of how far the current puzzle state is from the solution.

---

##### heuristic(state, goal)
**Purpose**: Calculates the Manhattan Distance heuristic between the current state and the goal.

**How it works**:
- For each tile in the grid (except 0), find its:
- Current position (x1, y1)
- Goal position (x2, y2)
- Compute the sum of abs(x1 - x2) + abs(y1 - y2) for all tiles.

In [90]:
def heuristic(state, goal):
    distance = 0
    for val in range(1, 6):  
        x1, y1 = np.where(state == val)
        x2, y2 = np.where(goal == val)
        distance += abs(x1[0] - x2[0]) + abs(y1[0] - y2[0])
    return distance

### Greedy Search

Greedy only considers the heuristic (Manhattan distance) when selecting the next state. It uses a priority queue and selects the state in the frontier with the lowest heuristic 

In [ ]:
# --- Greedy Implementation --- #

def greedy_search(initial_state, goal_state):
    start = state_to_tuple(initial_state)
    target = state_to_tuple(goal_state)
    
    if start == target:
        print("Greedy: Already at goal!")
        return 0

    # Initialise things 
    visited = set([start])
    frontier = []  # Priority queue for greedy (heuristic, state, path)
    heapq.heappush(frontier, (heuristic(initial_state, goal_state), start, [])) # (heuristic, state, path)
    states_explored = 0
    moves = 0
    
    while frontier:
        heuristic_value, state_tup, path = heapq.heappop(frontier)  # Pop state with lowest heuristic
        state = np.array(state_tup).reshape(2, 3)
        states_explored += 1
        
        # If the current state is the goal, print the solution path
        if state_tup == target:
            moves = len(path)
            print(f"Greedy solution found: {moves} moves")
            print(f"Greedy total states explored: {states_explored}")

            # Print the sequence of states and moves
            print_state(initial_state, title="Step 0: Initial State")
            for step, (action, s) in enumerate(path):
                print_state(s, title=f"Step {step + 1}: Move empty space {action}")
            return moves
        
        # Explore neighbors
        for action, neighbor_state in get_neighbors(state):
            tup = state_to_tuple(neighbor_state)
            
            if tup not in visited:
                visited.add(tup)
                heapq.heappush(frontier, (heuristic(neighbor_state, goal_state), tup, path + [(action, neighbor_state)]))
    
    # If no solution is found, return -1
    print("Greedy: Unsolvable puzzle.")
    print(f"Greedy: Total states explored: {states_explored}")
    return -1

print("\n===== SLIDING PUZZLE GREEDY TEST =====")
greedy_moves = greedy_search(INITIAL_STATE, GOAL_STATE)



===== SLIDING PUZZLE GREEDY TEST =====
Greedy solution found: 8 moves
Greedy total states explored: 9

Step 0: Initial State
5, 4, 2
1, 3,  

Step 1: Move empty space left
5, 4, 2
1,  , 3

Step 2: Move empty space up
5,  , 2
1, 4, 3

Step 3: Move empty space left
 , 5, 2
1, 4, 3

Step 4: Move empty space down
1, 5, 2
 , 4, 3

Step 5: Move empty space right
1, 5, 2
4,  , 3

Step 6: Move empty space up
1,  , 2
4, 5, 3

Step 7: Move empty space right
1, 2,  
4, 5, 3

Step 8: Move empty space down
1, 2, 3
4, 5,  


### A* Search 

A* search considered both the path cost and the heuristic when selecting the next state. It uses a priority queue and selects the state in the frontier with the lowest sum of path cost g(n) and heuristic h(n)

In [97]:
# --- A* Implementation --- #

def a_star_search(initial_state, goal_state):
    start = state_to_tuple(initial_state)
    target = state_to_tuple(goal_state)
    
    if start == target:
        print("A*: Already at goal!")
        return 0

    # Initialise things 
    visited = set([start])
    frontier = []  # Priority queue for A* (f(n), state, g(n), path)
    heapq.heappush(frontier, (heuristic(initial_state, goal_state), start, 0, []))  # (f(n), state, g(n), path)
    states_explored = 0
    moves = 0
    
    while frontier:
        f_value, state_tup, g_value, path = heapq.heappop(frontier)  # Pop state with lowest f(n) = g(n) + h(n)
        state = np.array(state_tup).reshape(2, 3)
        states_explored += 1
        
        # If the current state is the goal, print the solution path
        if state_tup == target:
            moves = len(path)
            print(f"A* solution found: {moves} moves")
            print(f"A* total states explored: {states_explored}")

            # Print the sequence of states and moves
            print_state(initial_state, title="Step 0: Initial State")
            for step, (action, s) in enumerate(path):
                print_state(s, title=f"Step {step + 1}: Move empty space {action}")
                moves += 1  # Add 1 for each step in the path
                
            return moves
        
        # Explore neighbors
        for action, neighbor_state in get_neighbors(state):
            tup = state_to_tuple(neighbor_state)
            
            if tup not in visited:
                visited.add(tup)
                g_new = g_value + 1  # Increment the cost from the start by 1 move
                f_new = g_new + heuristic(neighbor_state, goal_state)  # f(n) = g(n) + h(n)
                heapq.heappush(frontier, (f_new, tup, g_new, path + [(action, neighbor_state)]))
    
    # If no solution is found, return -1
    print("A*: Unsolvable puzzle.")
    print(f"A*: Total states explored: {states_explored}")
    return -1

# --- Run the Test --- #

print("\n===== SLIDING PUZZLE A* TEST =====")
a_star_moves = a_star_search(INITIAL_STATE, GOAL_STATE)



===== SLIDING PUZZLE A* TEST =====
A* solution found: 8 moves
A* total states explored: 9

Step 0: Initial State
5, 4, 2
1, 3,  

Step 1: Move empty space left
5, 4, 2
1,  , 3

Step 2: Move empty space up
5,  , 2
1, 4, 3

Step 3: Move empty space left
 , 5, 2
1, 4, 3

Step 4: Move empty space down
1, 5, 2
 , 4, 3

Step 5: Move empty space right
1, 5, 2
4,  , 3

Step 6: Move empty space up
1,  , 2
4, 5, 3

Step 7: Move empty space right
1, 2,  
4, 5, 3

Step 8: Move empty space down
1, 2, 3
4, 5,  


## Insights

| Algorithm   | # Moves in Solution | # States Explored |
|-------------|---------------------|-------------------|
| BFS         | 8                   | 53                |
| DFS         | 138                 | 182               |
| UCS         | 8                   | 49                |
| Greedy      | 8                   | 9                 |
| A*          | 8                   | 9                 |


**Optimal Solutions (#Moves in Solution):**
- BFS, UCS, Greedy, and A* are all able to find the optimal solution path in this case with (8 moves) 
- DFS, on the other hand, returns 138 moves, showing that DFS did not find the optimal path. This is typical of DFS's depth-first search nature, where it explores deep branches without considering the overall shortest path.

**Efficiency of Search (#States Explored):**

- DFS explores 182 states, much more than any of the other algorithms. This suggests that DFS may explore many unnecessary states in its search, leading to inefficiency. It may explore a long path to a dead end or revisit already explored states in different orders, contributing to its high state exploration.
- BFS and UCS explore 53 and 49 states respectively. The state count is reasonable since BFS explores all states layer by layer, and UCS explores states based on path cost. These two algorithms ensure an optimal solution is found, and hence will naturally explores a wider set of states than Greedy and A*. 
- Greedy and A* each explore 9 states, making them very efficient. This is due to the use of a good heuristic (Manhatten distance) that guides the search quickly towards the goal. Although they find the optimal solution in this case, in different puzzles, the heuristics might lead them to suboptimal paths. 